In [1]:
import pandas as pd
import numpy as np
from fancyimpute import IterativeSVD
from sklearn import svm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

Using TensorFlow backend.


In [5]:
data = pd.read_csv("train_with_missing/1.csv",index_col = False)
data.head(5)

,time,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13
0,0,231.455136,7.065539,NaN,176.942885,74.724240,19.164741,160.585702,196.760533,15.656928,21.489706,52.592477,3.937804,202.422745
1,225,233.443180,6.284800,36.515232,176.942885,69.607372,17.303101,160.585702,187.434032,19.769887,21.489706,53.611339,3.937804,242.045839
2,800,229.467092,6.909391,36.515232,173.675164,67.962665,17.569049,155.495242,213.340979,28.124336,22.059795,54.630201,3.780817,261.064923
3,1515,225.491003,6.597096,34.860942,172.585924,64.855995,16.505255,158.888882,207.123312,25.296676,22.344839,49.535892,3.309856,199.252898
4,3013,231.455136,6.597096,43.132392,179.121366,62.663051,15.973358,160.585702,195.724255,19.127237,22.202317,34.252965,2.838896,188.158431


In [6]:
data.isnull().sum(axis = 0)

time    0
X1      1
X2      1
X3      1
X4      1
X5      3
X6      3
X7      3
X8      3
X9      3
X10     3
X11     1
X12     1
X13     1
dtype: int64

In [7]:
values = data.values
values.shape

(17, 14)

In [9]:
X_filled_svd = IterativeSVD().fit_transform(values)

[IterativeSVD] Iter 1: observed MAE=47.098860
[IterativeSVD] Iter 2: observed MAE=9.093535
[IterativeSVD] Iter 3: observed MAE=2.080180
[IterativeSVD] Iter 4: observed MAE=0.381380
[IterativeSVD] Iter 5: observed MAE=0.043635


In [12]:
X_filled_svd

array([[0.00000000e+00, 2.31455136e+02, 7.06553901e+00, 4.06481436e+01,
        1.76942885e+02, 7.47242404e+01, 1.91647408e+01, 1.60585702e+02,
        1.96760533e+02, 1.56569277e+01, 2.14897064e+01, 5.25924773e+01,
        3.93780377e+00, 2.02422745e+02],
       [2.25000000e+02, 2.33443180e+02, 6.28480049e+00, 3.65152319e+01,
        1.76942885e+02, 6.96073723e+01, 1.73031005e+01, 1.60585702e+02,
        1.87434032e+02, 1.97698870e+01, 2.14897064e+01, 5.36113391e+01,
        3.93780377e+00, 2.42045839e+02],
       [8.00000000e+02, 2.29467092e+02, 6.90939131e+00, 3.65152319e+01,
        1.73675164e+02, 6.79626647e+01, 1.75690492e+01, 1.55495242e+02,
        2.13340979e+02, 2.81243356e+01, 2.20597948e+01, 5.46302009e+01,
        3.78081690e+00, 2.61064923e+02],
       [1.51500000e+03, 2.25491003e+02, 6.59709590e+00, 3.48609419e+01,
        1.72585924e+02, 6.48559948e+01, 1.65052547e+01, 1.58888882e+02,
        2.07123312e+02, 2.52966761e+01, 2.23448391e+01, 4.95358918e+01,
        3.309

In [42]:
y = values[:,13]
x = np.delete(X_filled_svd, 0, 1)

In [43]:
xtr,xt,ytr,yt=train_test_split(x, y, test_size=0.1, random_state=42)

In [48]:
clf = svm.SVC(gamma='scale')
X = np.array(xtr)
y = np.array(ytr)
from sklearn.svm import SVC
clf = SVC(gamma='auto')
X[np.isnan(X)] = 0
y[np.isnan(y)] = 0

In [73]:
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()

encoded_y = lab_enc.fit_transform(y)

print(utils.multiclass.type_of_target(y))
print(utils.multiclass.type_of_target(y.astype('int')))
print(utils.multiclass.type_of_target(encoded))

continuous
multiclass
multiclass


In [74]:
clf.fit(X, encoded)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [75]:
ytp = clf.predict(xt)
print(ytp)


[ 4 13]


In [76]:
from sklearn.metrics import mean_squared_error
print("SVM MSE = " , mean_squared_error(yt, ytp)*100 , "%")

SVM MSE =  4591679.0954252845 %


In [77]:
xtr[np.isnan(xtr)] = 0
ytr[np.isnan(ytr)] = 0
encoded_ytr = lab_enc.fit_transform(y)


gnb = GaussianNB()
gnb.fit(xtr,encoded_ytr)
ytp = gnb.predict(xt)
print("Naive bayes MSE = " , mean_squared_error(yt, ytp)*100 , "%")

Naive bayes MSE =  4591679.0954252845 %
